### Mount(Montar) Azure Data Lake mediante Service Principal
#### 1- Obtener el valor client_id, tenant_id y client_secret del key vault
#### 2- Configurar Spark con APP/Client ID, Directory/Tenant Id & Secret
#### 3- Utilizando el método "mount" de "utility" para montar el almacenamiento 
#### 4- Explorar otras utilidades del sistema de archivos relacionados con el montaje(list all mounts, unmounts)

In [0]:
# Prueba de conexión con el scope
client_id = dbutils.secrets.get(scope = "movie-history-secret-scope", key = "client-id")
tenant_id = dbutils.secrets.get(scope = "movie-history-secret-scope", key = "tenant-id")
secret_id = dbutils.secrets.get(scope = "movie-history-secret-scope", key = "client-secret")

In [0]:
def mount_adls(storage_account_name, container_name):
    # Obtener el secret key de Key Vault
    client_id = dbutils.secrets.get(scope = "movie-history-secret-scope", key = "client-id")
    tenant_id = dbutils.secrets.get(scope = "movie-history-secret-scope", key = "tenant-id")
    secret_id = dbutils.secrets.get(scope = "movie-history-secret-scope", key = "client-secret")

    # Establecer configuraciones de Spark
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": secret_id,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Desmontar si ya existe
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    # Mount(montar) el contenedor del Storage Account
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs
    )

    # Listar los Mount
    display(dbutils.fs.mounts())


In [0]:
mount_adls("moviehistory9","bronze")

In [0]:
mount_adls("moviehistory9","silver")

In [0]:
mount_adls("moviehistory9","gold")

#### Montar(mount) el contenedor "Demo"

In [0]:
mount_adls("moviehistory9","demo")